# 11点半
规则：
* 输入i，开始游戏
* 输入s，结束
* 输入y，抽取
* 输入exit，结束


In [71]:
from collections import namedtuple, defaultdict
Card = namedtuple("Card", ["suit", "rank"])
from enum import Enum
import random
import sys

class CardDeck:
    suits = list("♠♦♥♣")
    ranks = [str(i) for i in range(2, 11)] + list("JQKA")
    def __init__(self) -> None:
        print(self.suits)
        self._card = [Card(suit=x, rank=y) for x in CardDeck.suits for y in CardDeck.ranks]
    
    def __len__(self) -> int:
        return len(self._card)
    
    def __getitem__(self, pos) -> Card:
        return self._card[pos]
    
    def __setitem__(self, pos, other) -> None:
        self._card[pos] = other

    def __call__(self):
        if len(self) == 0:
            return None
        return self._card.pop()


from enum import Enum, auto

class GameState(Enum):
    INIT = auto()
    PLAYING = auto()
    STOP = auto()
    EXIT = auto()

accept_func = defaultdict(None)
accept_state = defaultdict(list)
display = defaultdict(str)
import functools
def state_machine_warp(*from_state, input_command, go_state):
    for i in from_state:
        accept_state[i].append(go_state)
    def decorator(f):
        global display
        display[input_command] = f.__doc__
        accept_func[input_command] = f
        @functools.wraps(f)
        def wrapper(self, *args, **kw):
            if all([self.state not in from_state, self.state != go_state]):
                return print("非预期的输入")
            try:
                f(self, *args, **kw)
                self.state = go_state
            except:
                self.state = GameState.STOP
        return wrapper
    return decorator

class ElevenHalf:
    state = GameState.INIT
    @staticmethod
    def default_score(): return 0.5
    card_score = defaultdict(default_score)
    card_score.update(dict({str(x): x for x in range(2, 11)}, A=1))
    def __init__(self) -> None:
        self.begin()

    @state_machine_warp(GameState.STOP, GameState.INIT, input_command = "i", go_state=GameState.PLAYING)
    def begin(self, seed = None):
        """start a new game"""
        self._card = CardDeck()
        if seed:
            random.shuffle(self._card)
        self._score = 0
        self._picked = []

    @state_machine_warp(GameState.PLAYING, input_command="y", go_state=GameState.PLAYING)
    def draw(self):
        """draw a card"""
        score = self._card()
        self._score += score
        if score >= 11.5:
            print("end")
            self.begin()
        self.record()

    @state_machine_warp(GameState.STOP, input_command="s", go_state=GameState.INIT)
    def record(self):
        """record"""
        print(f"score: {self._card}, picked: {self._picked}")
        return f"{self._score},{len(self._picked)}"
    
    @state_machine_warp(GameState.STOP, input_command="exit", go_state=GameState.EXIT)
    def exit(self):
        """exit"""
        pass

    def run(self):
        while(self.state != GameState.EXIT):
            for i in accept_state[self.state]:
                print(display[i])
            user_typing = input()
            if accept_func[user_typing] is None:
                print("Unexcepted Input!")
                continue
            accept_func[user_typing](accept_func[user_typing])


ElevenHalf().run()       




['♠', '♦', '♥', '♣']
